In [1]:
import torch
from diff3f import get_features_per_vertex
from time import time
from utils import convert_mesh_container_to_torch_mesh, cosine_similarity, double_plot, get_colors, generate_colors
from dataloaders.mesh_container import MeshContainer
from diffusion import init_pipe
from dino import init_dino
from functional_map import compute_surface_map
import os

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0.post101 with CUDA None (you have 2.1.0+cu121)
    Python  3.10.14 (you have 3.10.19)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [2]:
device = torch.device('cuda:0')
torch.cuda.set_device(device)
num_views = 100
H = 512
W = 512
num_images_per_prompt = 1
tolerance = 0.004
random_seed = 42
use_normal_map = True

In [3]:
def compute_features(device, pipe, dino_model, m, prompt):
    mesh = convert_mesh_container_to_torch_mesh(m, device=device, is_tosca=False)
    mesh_vertices = mesh.verts_list()[0]
    features = get_features_per_vertex(
        device=device,
        pipe=pipe, 
        dino_model=dino_model,
        mesh=mesh,
        prompt=prompt,
        mesh_vertices=mesh_vertices,
        num_views=num_views,
        H=H,
        W=W,
        tolerance=tolerance,
        num_images_per_prompt=num_images_per_prompt,
        use_normal_map=use_normal_map,
    )
    return features.cpu()

In [4]:
pipe = init_pipe(device)
os.environ["XFORMERS_DISABLED"] = '1'
dino_model = init_dino(device)

/home/junhokim/miniconda3/envs/diff3f/lib/python3.10/site-packages/diffusers/configuration_utils.py:239: FutureWarning: It is deprecated to pass a pretrained model name or path to `from_config`.If you were trying to load a model, please use <class 'unet_2d_condition.UNet2DConditionModel'>.load_config(...) followed by <class 'unet_2d_condition.UNet2DConditionModel'>.from_config(...) instead. Otherwise, please make sure to pass a configuration dictionary instead. This functionality will be removed in v1.0.0.
  deprecate("config-passed-as-path", "1.0.0", deprecation_message, standard_warn=False)


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'pipeline_controlnet_img2img.StableDiffusionControlNetImg2ImgPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .
Using cache found in /home/junhokim/.cache/torch/hub/facebookresearch_dinov2_main
/home/junhokim/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:45: UserWarning: xFormers is disabled (SwiGLU)
  warnings.warn("xFormers is disabled (SwiGLU)")
/home/junhokim/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWa

In [17]:
source_file_path = "meshes/chair_1_rot_45.obj"
target_file_path = "meshes/chair_2.obj"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)

In [18]:
f_source = compute_features(device, pipe, dino_model, source_mesh, "chair")

Rendering complete


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:17<00:00,  3.77s/it]

Number of missing features:  620
Copied features from nearest vertices
Time taken in mins:  6.54892239967982


In [8]:
f_target = compute_features(device, pipe, dino_model, target_mesh, "chair")

Rendering complete


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:20<00:00,  3.80s/it]

Number of missing features:  4
Copied features from nearest vertices
Time taken in mins:  6.365933227539062


In [15]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]

In [16]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Apply functional map on features

In [20]:
source_file_path = "meshes/cat.off"
target_file_path = "meshes/lion.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "cat")
f_target = compute_features(device, pipe, dino_model, target_mesh, "lion")

Rendering complete


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:05<00:00,  3.66s/it]


Number of missing features:  140
Copied features from nearest vertices
Time taken in mins:  6.495763293902079
Rendering complete


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [06:11<00:00,  3.72s/it]

Number of missing features:  17
Copied features from nearest vertices
Time taken in mins:  6.476000332832337


In [21]:
s = cosine_similarity(f_source.to(device),f_target.to(device))
s = torch.argmax(s, dim=0).cpu().numpy()
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[s]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [11]:
surface_map = compute_surface_map(source_file_path, target_file_path, f_source.numpy(), f_target.numpy())

mesh1 (4638, 3)
mesh2 (259, 3)

Computing Laplacian spectrum
Computing 200 eigenvectors
	Done in 2.12 s
Computing 200 eigenvectors
Problem during LBO decomposition ! Please check


/home/junhokim/Projects/Diffusion-3D-Features/pyFM/mesh/laplacian.py:121: RuntimeWarning: divide by zero encountered in divide
  S = 0.5 * S / np.sqrt(1-S**2)
/home/junhokim/Projects/Diffusion-3D-Features/pyFM/mesh/laplacian.py:151: UserWarning: The problem size 259 minus the constraints size 0 is too small relative to the block size 200. Using a dense eigensolver instead of LOBPCG iterations.No output of the history of the iterations.
  eigenvalues, eigenvectors = sparse.linalg.lobpcg(W, init_eigenvecs,


Exception: Dense eigensolver failed with error
The leading minor of order 258 of B is not positive definite. The factorization of B could not be completed and no eigenvalues or eigenvectors were computed.


In [24]:
cmap_source = get_colors(source_mesh.vert); cmap_target = cmap_source[surface_map.cpu().numpy()]
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

# Part segmentation

In [29]:
from sklearn.cluster import KMeans
import numpy as np

k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner
segments2 = kmeans.predict(f_target)

In [30]:
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [31]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  

In [39]:
source_file_path = "meshes/posed_human.off"
target_file_path = "meshes/cat.off"
source_mesh = MeshContainer().load_from_file(source_file_path)
target_mesh = MeshContainer().load_from_file(target_file_path)
f_source = compute_features(device, pipe, dino_model, source_mesh, "naked human")
f_target = compute_features(device, pipe, dino_model, target_mesh, "cat")

Rendering complete


100%|██████████| 100/100 [03:13<00:00,  1.93s/it]


Number of missing features:  3
Time taken in mins:  3.3452319582303365
Rendering complete


100%|██████████| 100/100 [03:17<00:00,  1.97s/it]

Number of missing features:  140
Time taken in mins:  3.4606642444928486


In [40]:
k = 6

kmeans = KMeans(n_clusters=k, random_state=0, n_init="auto").fit(f_source)

segments1 = kmeans.predict(f_source)

# Apply centroids on another mesh to segment it in a corresponding manner

segments2 = kmeans.predict(f_target)
segment_colors = generate_colors(k)
cmap_source = np.array([segment_colors[j] for j in segments1])
cmap_target = np.array([segment_colors[j] for j in segments2])

In [41]:
double_plot(source_mesh,target_mesh,cmap_source,cmap_target)  